Install Gurobi

In [1]:
# !pip install gurobipy
import gurobipy as gp
import pandas as pd
import numpy as np
from gurobipy import *
Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-03


<gurobi.Model Continuous instance Unnamed: 0 constrs, 0 vars, Parameter changes: Username=(user-defined)>

In [2]:
def DataEngineering(C,I,destination):
    consoles_temp = C[['Id','Max Vol','Height','Width','Depth','Technical Max Weight','Pivot Weight','Rate to Pivot Weight','Rate Above Pivot Weight','Fix Rate']]
    consoles_temp = consoles_temp.rename(columns={'Id':'id','Max Vol':'max_vol','Technical Max Weight':'tech_max_weight','Pivot Weight':'pivot_weight',
                                        'Rate to Pivot Weight':'rate_2_pivot_weight','Rate Above Pivot Weight':'rate_above_pivot_weight','Fix Rate':'fix_rate'})
    consoles_temp['id'] = consoles_temp['id'].astype(str)
    
    consoles_temp['max_dimension'] = 0
    for _,row in consoles_temp.iterrows():
        t = [row['Height'],row['Depth'],row['Width']]
        consoles_temp['max_dimension'].loc[_] = max(t)
        if pd.isnull(row['pivot_weight']):
            consoles_temp['pivot_weight'].loc[_] = consoles_temp['tech_max_weight'].loc[_]
            consoles_temp['rate_2_pivot_weight'].loc[_] = consoles_temp['fix_rate'].loc[_]
            consoles_temp['rate_above_pivot_weight'].loc[_] = consoles_temp['fix_rate'].loc[_]
    
    consoles_temp = consoles_temp.drop(columns='fix_rate',axis=1)
    
    consoles_temp = consoles_temp.drop(axis=1,columns=['Height','Width','Depth'])
    
    consoles_temp.to_excel('./Data/OutputData/cleanedConsoles'+str(destination)+'.xlsx',index=False)
    
    #Convert Dataframes to Dictionary
    consoles_dict = consoles_temp.set_index('id').T.to_dict('list')
    
    items_temp = I[['id_x','weight','height','length','width']]
    items_temp['volume'] = items_temp['height']*items_temp['length']*items_temp['width']
    #items = items.drop(columns=['height','length','width'],axis=1)
    dict = {}
    for index, row in items_temp.iterrows():
        x,y,z = row['length'],row['width'],row['height']
        l = [x,y,z]
        l.sort(reverse=True)
        d = {row['id_x'] : l}
        dict.update(d)
    
    df1 = pd.DataFrame.from_dict(dict, orient='index').rename(columns={0:'width',1:'length',2:'height'})
    df1['id_x'] = df1.index
    
    items_temp = items_temp.drop(axis=1,columns=['height','length','width'])
    
    items_temp = items_temp.merge(df1, on = ['id_x'], how='inner')
    items_temp['id'] = 0
    
    for i,ele in enumerate(items_temp['id_x']):
        x=ele.split('_')
        items_temp['id'].iloc[i]=x[0]
    
    items_temp = items_temp.merge(items_temp[['id','weight']].groupby(by="id").mean(),on='id',how='left')
    items_temp = items_temp[['id_x','weight_y','volume','width']]
    items_temp.to_excel('./Data/OutputData/cleanedItems'+str(destination)+'.xlsx',index=False)
    items_dict = items_temp.set_index('id_x').T.to_dict('list')

    return consoles_dict,items_dict

Create an optimization model

In [3]:

def printSolution(model,console_id,item_id,I2C):
    itemId = []
    consoleId = []
    Xij = []
    print('\nCost: %g' % model.ObjVal)
    print('\nItem Assignment:')
    print('\nItem_id Container_Id')
    for j in console_id:
        for i in item_id:
            if I2C[i,j].X > 0.0001:
                itemId.append(i)
                consoleId.append(j)
                Xij.append(I2C[i,j].X)
                print('%s %s %g' % (i,j, I2C[i,j].X))
    
    return itemId,consoleId,Xij,model.ObjVal

In [4]:
def optimize(items,consoles,destination):
    
    console_id,max_vol,tech_max_weight,pivot_weight,rate_2_pivot_weight,rate_above_pivot_weight,max_dimension_console = gp.multidict(consoles)
    item_id, item_weight, item_volume, max_dimension_item = gp.multidict(items)

    model = Model("Item assignment model")

    M = 100000

    ## Start Objective ##
    ## Add variables ##

    #Decision variable to define the total weight of all the items in a container
    TOT_W_C = model.addVars(console_id,vtype = gp.GRB.CONTINUOUS,name = "TOT_W_C")
    #Binary decision variable to decide if container j has exceeded pivot weight or not
    #X_Pivot = model.addVars(console_id,vtype = gp.GRB.BINARY,name = "X_Pivot")
    #Binary decision variable to decide if item i goes into container j or not
    I2C = model.addVars(item_id,console_id,vtype = gp.GRB.BINARY,name = "I2C")

    X_Pivot = {}  # Assuming X_Pivot is a dictionary
    for j in console_id:
        X_Pivot[j] = model.addVar(vtype=gp.GRB.BINARY, name=f"X_Pivot_{j}")

    model.update()

    #add constraints

    #Every item needs to packed once
    model.addConstrs((gp.quicksum(I2C[i,j] for j in console_id) == 1 for i in item_id), "_")
    #Total Weight of the items in a container
    model.addConstrs((gp.quicksum(I2C[i,j]*item_weight[i] for i in item_id) == TOT_W_C[j] for j in console_id),"_")
    #Weight capacity constraint
    model.addConstrs((gp.quicksum(I2C[i,j]*item_weight[i] for i in item_id) <= tech_max_weight[j] for j in console_id),"_")
    #Volume capacity constraint
    model.addConstrs((gp.quicksum(I2C[i,j]*item_volume[i] for i in item_id) <= 0.9*max_vol[j] for j in console_id),"_")
    #Check if the weight of all the items in container j exceeds pivot weight
    model.addConstrs((M*X_Pivot[j] + pivot_weight[j] >= TOT_W_C[j] for j in console_id),"_")
    #Check to avoid overfitting
    #model.addConstrs(((max_dimension_item[i]*I2C[i,j] for i in item_id) <= max_dimension_console[j] for j in console_id), "_")

    model.update()

    'objective'
    # pivot_sum = gp.quicksum(rate_2_pivot_weight[j]*pivot_weight[j] for j in console_id)
    # above_pivot_sum = gp.quicksum(rate_above_pivot_weight[j]*(TOT_W_C[j] - pivot_weight[j]) for j in console_id)
    #Set objective function
    # model.setObjective(pivot_sum + above_pivot_sum, gp.GRB.MINIMIZE)

    # Define objective terms
    pivot_sum = {}
    above_pivot_sum = {}
    pivot_sum_below = {}

    for j in console_id:
        if j in rate_2_pivot_weight and j in pivot_weight:
            pivot_sum[j] = rate_2_pivot_weight[j] * pivot_weight[j]
        else:
            pivot_sum[j] = 0.0

        if j in rate_above_pivot_weight and j in TOT_W_C and j in pivot_weight:
            above_pivot_sum[j] = rate_above_pivot_weight[j] * (TOT_W_C[j] - pivot_weight[j])
        else:
            above_pivot_sum[j] = 0.0

        if j in rate_2_pivot_weight and j in TOT_W_C:
            pivot_sum_below[j] = rate_2_pivot_weight[j] * TOT_W_C[j]
        else:
            pivot_sum_below[j] = 0.0

    # Set objective function
    expr = gp.quicksum(pivot_sum[j] * X_Pivot[j] + above_pivot_sum[j] * X_Pivot[j] for j in console_id) \
           + gp.quicksum(pivot_sum_below[j] * (1 - X_Pivot[j]) for j in console_id)

    # Set the objective function
    model.setObjective(expr, gp.GRB.MINIMIZE)

    model.setParam('TimeLimit', 20*60)
    model.optimize()

    itemId, consoleId, Xij, objVal = printSolution(model,console_id,item_id,I2C)

    solution = pd.DataFrame(list(zip(itemId,consoleId,Xij)),columns=["item_id","console_id","x_ij"])
    solution.to_excel('./Data/OutputData/OptimisationSolutionFor'+ str(destination) +'.xlsx',index=False)

    return objVal

In [5]:
consoles = pd.read_excel('./Data/InputData/ConsoleData.xlsx')
consoles.head()

items = pd.read_csv('./Data/InputData/itemsList.csv')
items.head()

destinations = consoles['Destination'].unique().tolist()

objective_value = 0

for destination in destinations:
    
    consoles_t = consoles[consoles['Destination'] == destination]
    items_t = items[items['destination'] == destination]

    consoles_dict,items_dict= DataEngineering(consoles_t,items_t,destination)

    objective_value += optimize(items_dict,consoles_dict,destination)

print('The total objective value is ',objective_value)

C:\Users\anike\AppData\Local\Temp\ipykernel_14724\3233626228.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  consoles_temp['max_dimension'].loc[_] = max(t)
C:\Users\anike\AppData\Local\Temp\ipykernel_14724\3233626228.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  items_temp['volume'] = items_temp['height']*items_temp['length']*items_temp['width']
C:\Users\anike\AppData\Local\Temp\ipykernel_14724\3233626228.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pan

Set parameter TimeLimit to value 1200
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 6800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 785 rows, 17175 columns and 68575 nonzeros
Model fingerprint: 0x8ba70640
Model has 25 quadratic objective terms
Variable types: 25 continuous, 17150 integer (17150 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+06]
  Objective range  [3e+01, 6e+04]
  QObjective range [6e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+07]
Found heuristic solution: objective 1834917.0099
Presolve time: 0.05s
Presolved: 810 rows, 17200 columns, 51550 nonzeros
Variable types: 50 continuous, 17150 integer (17150 binary)

Root relaxation: objective 1.427206e+06, 8881 iterations, 0.12 seconds (0.15 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Une

C:\Users\anike\AppData\Local\Temp\ipykernel_14724\3233626228.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  consoles_temp['max_dimension'].loc[_] = max(t)
C:\Users\anike\AppData\Local\Temp\ipykernel_14724\3233626228.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  consoles_temp['pivot_weight'].loc[_] = consoles_temp['tech_max_weight'].loc[_]
C:\Users\anike\AppData\Local\Temp\ipykernel_14724\3233626228.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 502589.923    0   11 513718.127 502589.923  2.17%     -    0s
H    0     0                    502972.63328 502589.923  0.08%     -    0s
H    0     0                    502966.75828 502589.923  0.07%     -    0s
H    0     0                    502932.79472 502589.923  0.07%     -    0s
     0     0 502589.923    0   22 502932.795 502589.923  0.07%     -    0s
H    0     0                    502609.44823 502589.923  0.00%     -    0s
     0     0 502589.923    0   22 502609.448 502589.923  0.00%     -    0s

Cutting planes:
  Gomory: 2
  Cover: 5
  Clique: 10
  MIR: 2
  StrongCG: 3
  RLT: 1

Explored 1 nodes (385 simplex iterations) in 0.09 seconds (0.02 work units)
Thread count was 16 (of 16 available processors)

Solution count 6: 502609 502933 502967 ... 524189

Optimal solution found (tolerance 1.00e-04)
Best objective 5.

C:\Users\anike\AppData\Local\Temp\ipykernel_14724\3233626228.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  consoles_temp['max_dimension'].loc[_] = max(t)
C:\Users\anike\AppData\Local\Temp\ipykernel_14724\3233626228.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  items_temp['volume'] = items_temp['height']*items_temp['length']*items_temp['width']
C:\Users\anike\AppData\Local\Temp\ipykernel_14724\3233626228.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pan

...

C:\Users\anike\AppData\Local\Temp\ipykernel_14724\3233626228.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  consoles_temp['max_dimension'].loc[_] = max(t)
C:\Users\anike\AppData\Local\Temp\ipykernel_14724\3233626228.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  consoles_temp['pivot_weight'].loc[_] = consoles_temp['tech_max_weight'].loc[_]
C:\Users\anike\AppData\Local\Temp\ipykernel_14724\3233626228.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu

Set parameter TimeLimit to value 1200
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 6800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 658 rows, 21296 columns and 84964 nonzeros
Model fingerprint: 0x64e06432
Model has 24 quadratic objective terms
Variable types: 44 continuous, 21252 integer (21252 binary)
Coefficient statistics:
  Matrix range     [5e-01, 8e+06]
  Objective range  [2e+01, 1e+05]
  QObjective range [1e+01, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+07]
Found heuristic solution: objective 1895070.7071
Presolve removed 64 rows and 106 columns
Presolve time: 0.11s
Presolved: 618 rows, 21214 columns, 63570 nonzeros
Variable types: 48 continuous, 21166 integer (21166 binary)
Found heuristic solution: objective 1889411.8160

Root relaxation: objective 1.352159e+06, 16659 iterations, 0.29 seconds (0.42 work

C:\Users\anike\AppData\Local\Temp\ipykernel_14724\3233626228.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  consoles_temp['max_dimension'].loc[_] = max(t)
C:\Users\anike\AppData\Local\Temp\ipykernel_14724\3233626228.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  consoles_temp['pivot_weight'].loc[_] = consoles_temp['tech_max_weight'].loc[_]
C:\Users\anike\AppData\Local\Temp\ipykernel_14724\3233626228.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu

Set parameter TimeLimit to value 1200
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 6800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 664 rows, 21242 columns and 84753 nonzeros
Model fingerprint: 0x224d2b80
Model has 21 quadratic objective terms
Variable types: 43 continuous, 21199 integer (21199 binary)
Coefficient statistics:
  Matrix range     [5e-01, 7e+06]
  Objective range  [3e+01, 6e+04]
  QObjective range [1e+01, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+08]
Found heuristic solution: objective 2560229.6190
Presolve removed 66 rows and 44 columns
Presolve time: 0.12s
Presolved: 619 rows, 21219 columns, 63102 nonzeros
Variable types: 42 continuous, 21177 integer (21177 binary)
Found heuristic solution: objective 2489162.0939

Root relaxation: objective 1.843418e+06, 2324 iterations, 0.10 seconds (0.16 work u

C:\Users\anike\AppData\Local\Temp\ipykernel_14724\3233626228.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  consoles_temp['max_dimension'].loc[_] = max(t)
C:\Users\anike\AppData\Local\Temp\ipykernel_14724\3233626228.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  consoles_temp['pivot_weight'].loc[_] = consoles_temp['tech_max_weight'].loc[_]
C:\Users\anike\AppData\Local\Temp\ipykernel_14724\3233626228.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu